In [1]:
from mhkit import acoustics

P = acoustics.io.read_audiofile("data/acoustics/RBW_6661_20240601_053114.wav", Sf=100)

In [2]:
P

array([31897.6, 30566.4, 32588.8, ...,  8601.6,  7372.8,  6348.8])